In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pytz
import datetime
import orchest

from bs4 import BeautifulSoup
import requests

In [10]:
def construct_url(page=1):
    tweakers_url = "https://gathering.tweakers.net/forum/find/poster/{poster}/messages?data%5Bpage%5D={page}"
    return tweakers_url.format(page=page, poster=orchest.get_step_param("poster_id"))

In [11]:
tz = pytz.timezone('Europe/Amsterdam')

In [12]:
def fetch_comments_from_page(page=1):
    comments = []
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    resp = requests.get(construct_url(page), headers=headers)
    soup = BeautifulSoup(resp.content, features="html.parser")

    for comment in soup.select(".message"):
        unix_timestamp = comment.select("a > span")[0]['data-timestamp']
        for bq in comment.select("blockquote"):
            bq.decompose()
        timestamp = datetime.datetime.fromtimestamp(int(unix_timestamp), tz=tz).isoformat()
        comment_text = comment.select(".messagecontent")[0].text
        comment_link = comment.select(".messageheader a")[0]['href']
        comments.append({"content": comment_text, "date": timestamp, "permalink": comment_link})
    return comments

In [13]:
page_count = orchest.get_step_param("pages")

In [14]:
comments = []

for page in range(page_count):
    page = page + 1
    print("Fetching page %d" % page)
    new_comments = fetch_comments_from_page(page)
    print("Found %d comments." % len(new_comments))
    
    if len(new_comments) == 0:
        print("Found no new comments, stopping page traversal.")
        break
    comments += new_comments

Fetching page 1
Found 30 comments.
Fetching page 2
Found 0 comments.
Found no new comments, stopping page traversal.


In [15]:
f"Fetched {len(comments)} comments"

'Fetched 30 comments'

In [16]:
comments[0:3]

[{'content': 'Ik zou persoonlijk ook een 3700x combineren met een 2080 Ti. Dus wat @ECP0WERS voorstelt.\n\r\nMooi budget! Veel plezier met de build ',
  'date': '2019-09-23T03:33:46+02:00',
  'permalink': 'https://gathering.tweakers.net/forum/list_message/60087504#60087504'},
 {'content': "Leuk systeem heb je al samengesteld.\n\r\nVoor deep learning is denk ik 16GB ram wellicht wat aan de lage kant. Je zult merken dat voor grote datasets het in memory kunnen laden van je data het trainproces enorm kan versnellen.\n\r\nOok heeft de 2070 'maar' 8GB VRAM. Als je meerdere of grotere modellen wil instantiëren kun je hier tegen limitaties aanlopen. Dan is misschien de 2080 Ti beter (natuurlijk tegen een forse meerprijs).\n\r\nVerder had ik nog gekeken naar je moederbord. Met 2700X krijg je 24 PCI lanes en zal je met dual GPU dus niet op x16/x16 kunnen draaien. Al zag ik in een analyse van Puget dat dat geen ramp hoeft te zijn voor performance. https://www.pugetsystems....or-Machine-Learning-

In [17]:
orchest.output(comments, name="comments")